In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读数据

In [2]:
path = 'process data/dataset3/'
df = pd.read_csv(path + 'dataset_3.csv')
df

,device_id,latitude,longitude
0,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445384,114.035978
1,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445352,114.035938
2,ec11d950-75e3-4d22-b27f-4e7203f94342,22.364642,114.136940
3,96c515c6-540f-45d3-9c9b-ff4edb701896,22.384730,114.196530
4,96e87f08-a206-4ba9-aaed-395d6495aecc,22.365936,114.057621
...,...,...,...
164376,931adaea-055c-4203-bba9-8c08786277cc,22.501801,114.129160
164377,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424879,114.229265
164378,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424917,114.229258
164379,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424910,114.229247


In [5]:
device_cnt = np.size(df['device_id'].unique())
device_cnt
## 可以推测住址的用户有 31126 个

31126

In [4]:
## 转换为 GeoDataFrame
geometry = gpd.points_from_xy(df['longitude'], df['latitude'])
location_lat_lon_gdf = gpd.GeoDataFrame(df, geometry=geometry)
location_lat_lon_gdf.crs = 'EPSG:4326'
location_lat_lon_gdf = location_lat_lon_gdf.drop(['longitude','latitude'], axis=1)
location_lat_lon_gdf

,device_id,geometry
0,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03598 22.44538)
1,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03594 22.44535)
2,ec11d950-75e3-4d22-b27f-4e7203f94342,POINT (114.13694 22.36464)
3,96c515c6-540f-45d3-9c9b-ff4edb701896,POINT (114.19653 22.38473)
4,96e87f08-a206-4ba9-aaed-395d6495aecc,POINT (114.05762 22.36594)
...,...,...
164376,931adaea-055c-4203-bba9-8c08786277cc,POINT (114.12916 22.50180)
164377,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22927 22.42488)
164378,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22926 22.42492)
164379,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22925 22.42491)


## 读building shape file

In [3]:
shapefile = gpd.read_file('Building_SHP/BUILDING_STRUCTURE.shp', encoding="GBK")
shapefile = shapefile.to_crs('EPSG:4326')
building_shp = shapefile[['OBJECTID', 'geometry']]
building_shp

,OBJECTID,geometry
0,1,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,2,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,3,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,4,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,5,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...
340747,340748,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,340749,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,340750,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,340751,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


## 关联 building_shp 和 location_lat_lon_gdf  #merged

In [6]:
merged = gpd.sjoin(building_shp, location_lat_lon_gdf, how='inner', predicate='intersects')
merged = merged.drop('index_right', axis=1)
merged

,OBJECTID,geometry,device_id
494,495,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,495,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,495,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,495,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",cb70e6e9-b2ce-6a70-becb-b807e32c87de
494,495,"POLYGON ((113.86227 22.25378, 113.86222 22.253...",cb70e6e9-b2ce-6a70-becb-b807e32c87de
...,...,...,...
338157,338158,"POLYGON ((114.30081 22.39230, 114.30076 22.392...",5d00dd96-3882-4b0e-b2b8-20d51cc37424
338798,338799,"POLYGON ((114.31586 22.39473, 114.31579 22.394...",b34c4153-d1a9-4cc7-8d1b-d912704db125
338798,338799,"POLYGON ((114.31586 22.39473, 114.31579 22.394...",b34c4153-d1a9-4cc7-8d1b-d912704db125
338798,338799,"POLYGON ((114.31586 22.39473, 114.31579 22.394...",b34c4153-d1a9-4cc7-8d1b-d912704db125


In [10]:
device_building_loc_df = merged[['device_id','OBJECTID']]
device_building_loc_df

,device_id,OBJECTID
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,495
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,495
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,495
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,495
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,495
...,...,...
338157,5d00dd96-3882-4b0e-b2b8-20d51cc37424,338158
338798,b34c4153-d1a9-4cc7-8d1b-d912704db125,338799
338798,b34c4153-d1a9-4cc7-8d1b-d912704db125,338799
338798,b34c4153-d1a9-4cc7-8d1b-d912704db125,338799


## 推测住址

In [11]:
## 计算每个 device_id 的每个 OBJECTID 出现的次数
counts = device_building_loc_df.groupby(['device_id','OBJECTID']).size()
counts

device_id                             OBJECTID
0002d2d4-b519-4255-9167-b6c7a9fa9f5b  179801      4
0014bb13-b820-4646-accb-08f9877d4c9c  228353      1
                                      229059      1
001ba349-ab7b-4c7c-b796-fc3bd6751660  215566      2
0020bd39-a85b-489f-825c-ff9d6b0cf92a  226806      1
                                                 ..
ffbe74d6-fea8-46d0-adc7-229938ad6daf  194737      2
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6  16749       2
ffdb8998-0ca7-48e2-9fa3-339de0094167  211114      1
                                      211788      1
fffc482d-4e69-48b6-a765-9c4468f60d90  208300      8
Length: 8973, dtype: int64

In [12]:
## 保存 device_id 和 这个OBJECTID出现次数最多的 OBJECTID
max_OBJECTID = counts.groupby('device_id').idxmax().apply(lambda x: x[1])
max_OBJECTID

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b    179801
0014bb13-b820-4646-accb-08f9877d4c9c    228353
001ba349-ab7b-4c7c-b796-fc3bd6751660    215566
0020bd39-a85b-489f-825c-ff9d6b0cf92a    226806
0020e82b-70c2-444b-98c4-88bf78afcc60    217799
                                         ...  
ffb07629-c7f3-41c1-a530-458095912380    249212
ffbe74d6-fea8-46d0-adc7-229938ad6daf    194737
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6     16749
ffdb8998-0ca7-48e2-9fa3-339de0094167    211114
fffc482d-4e69-48b6-a765-9c4468f60d90    208300
Length: 5960, dtype: int64

In [24]:
f_counts = counts[counts > 1] ## 剔除只出现过一次的building
f_counts.groupby('device_id').idxmax().apply(lambda x: x[1])

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b    179801
001ba349-ab7b-4c7c-b796-fc3bd6751660    215566
0081143a-d9c7-40be-9d8c-28ada9767c60    186331
009ce6a0-db40-6912-b8f3-99346bf8b797    160973
00a60570-4d55-4e3d-a5ff-3977f96d628f    247401
                                         ...  
ff07b1b6-bee5-47ef-b6fb-f1c4f6cf11d7    285149
ff2ce447-7363-4f95-9734-697671932b5c    263724
ffbe74d6-fea8-46d0-adc7-229938ad6daf    194737
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6     16749
fffc482d-4e69-48b6-a765-9c4468f60d90    208300
Length: 2882, dtype: int64

In [13]:
location_building_pred_df = pd.DataFrame({'device_id': max_OBJECTID.index, 'OBJECTID': max_OBJECTID.values})
location_building_pred_df

## 31126 个用户中， 成功推测出住址的有 5960 个

,device_id,OBJECTID
0,0002d2d4-b519-4255-9167-b6c7a9fa9f5b,179801
1,0014bb13-b820-4646-accb-08f9877d4c9c,228353
2,001ba349-ab7b-4c7c-b796-fc3bd6751660,215566
3,0020bd39-a85b-489f-825c-ff9d6b0cf92a,226806
4,0020e82b-70c2-444b-98c4-88bf78afcc60,217799
...,...,...
5955,ffb07629-c7f3-41c1-a530-458095912380,249212
5956,ffbe74d6-fea8-46d0-adc7-229938ad6daf,194737
5957,ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6,16749
5958,ffdb8998-0ca7-48e2-9fa3-339de0094167,211114


In [14]:
## 保存数据：device_building_location
location_building_pred_df.to_csv(path + "device_building_location.csv", index=False)